In [13]:
import os
import numpy as np
import cv2

In [14]:
files = os.listdir()
d = {}
cnt = 0 

X = []
Y = []
for f in files:
    if f.endswith(".npy"):
        data = np.load(f)
        print(data.shape,f)
        labels = np.ones((data.shape[0],1))*cnt
        
        X.append(data)
        Y.append(labels)
        d[cnt] = f[:-4]
        cnt += 1

(18, 100, 100, 3)  Ausho.npy
(20, 100, 100, 3) abdur.npy


In [15]:
files = os.listdir()
files

[' Ausho.npy',
 '.ipynb_checkpoints',
 'abdur.npy',
 'bac.png',
 'Datasets',
 'FaceDataCollection.ipynb',
 'FaceRecognitionMain.ipynb',
 'fake_ai_faces.0.png',
 'haarcascade_frontalface_alt.xml',
 'HandWrittenDigitRecognition.ipynb',
 'image.jpg',
 'Matplotlib.ipynb',
 'Numpy Basics.ipynb',
 'Numpy.ipynb',
 'OpenCVandFaceDetection.ipynb',
 'pokemon.jpg',
 'PokemonClassifier.ipynb',
 'PythonBasics.ipynb',
 'Python_Basics.ipynb',
 'Train']

In [16]:
d

{0: ' Ausho', 1: 'abdur'}

In [19]:
X = np.vstack(X)
Y = np.vstack(Y)
Y = Y.reshape((38,))
X

array([[[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       [[  0, 255,   0],
        [ 63, 116,  48],
        [ 65,  95,  51],
        ...,
        [112, 156, 116],
        [121, 165, 125],
        [ 36, 229,  38]],

       [[  0, 255,   0],
        [ 74, 100,  57],
        [ 76,  76,  61],
        ...,
        [130, 147, 134],
        [137, 155, 143],
        [ 40, 226,  43]],

       ...,

       [[  0, 255,   0],
        [ 50, 162,  42],
        [ 99,  86,  82],
        ...,
        [116, 112, 102],
        [157, 155, 147],
        [ 32, 241,  30]],

       [[  0, 255,   0],
        [ 48, 161,  41],
        [ 95,  84,  81],
        ...,
        [138, 138, 128],
        [182, 186, 178],
        [ 34, 244,  34]],

       [[  0, 255,   0],
        [  9, 238,   7],
        [ 17, 224,  14],
        ...,
        [ 28, 237,  26],
        [ 34, 244,  33],
        [  6, 253,   6]]

In [18]:
print(X.shape,Y.shape)

(38, 100, 100, 3) (38,)


In [7]:
def dist(v1, v2):
    return np.sqrt(np.sum((v1-v2)**2))

def knn(X, Y, query_point, k=7):
    m = X.shape[0]
    distances = []
    for i in range(m):
        d = dist(query_point,X[i])
        distances.append((d,Y[i]))
    
    distances = sorted(distances)[:k]
    distances = np.array(distances)
    labels = distances[:,1]
    labels, freq =np.unique(labels, return_counts=True)
    pred =labels[np.argmax(freq)]
    
    return int(pred)


In [8]:
cam = cv2.VideoCapture(0)
detector = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")



while True:
    ret, frame = cam.read()
    
    if ret == False:
        continue
    
    faces = detector.detectMultiScale(frame, 1.2)
    if(len(faces)==0):
        continue
    
    for face in faces:
        
        x, y, w, h = face
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0),2)
        cropped_face = frame[y:y+h,x:x+w]
        cropped_face = cv2.resize(cropped_face,(100,100))
        #Find out the name of the face
        pred = knn(X,Y,cropped_face,k=5)
        pred_name = d[pred]
        cv2.putText(frame, pred_name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), lineType=cv2.LINE_AA)
         
    cv2.imshow("frame",frame)
    
    key_pressed  = cv2.waitKey(1) & 0xFF
    
    if(key_pressed == ord('q')):
        break
    
cam.release()
cv2.destroyAllWindows()
